In [1]:
import os
import sys

In [2]:
#sys.path.insert(0, '/Users/marusy/Programming/another-fem-framework/')
sys.path.insert(0, '/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0, '/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [3]:
import numpy as np
from interpolant.cell_to_points import Cell2PointsConverter
from interpolant.strict import StrictSolve
from interpolant.opt import OptSolve
from common.helpers import *
from common.tests import *
from interpolant.helpers import *
from matplotlib import pyplot as plt
from common import polynom_factory
import h5py
import sympy
%matplotlib inline

In [4]:
order = 5

#glob_2d_weights = polynom_factory.local_funcfunc_matrix(dim=2,distribution='globatto', order=order).sum(axis=1)
points = polynom_factory.glob_points(order=order, size=(0,1))
points_2d = list(itertools.product(points, repeat=2))

In [5]:
gradgrad = polynom_factory.local_gradgrad_functions(dim=2, distribution='globatto', order=order)
grads = polynom_factory.local_gradients(dim=2, distribution='globatto', order=order)

In [15]:
matrix = np.zeros(tuple(2*[len(grads)]))
sub_dict = [{sym: val for sym, val in zip(gradgrad[1], point)} for point in points_2d]
for num, (ind, gg) in enumerate(gradgrad[0].items()):
    try:
        lmb = sympy.lambdify(gg.free_symbols, gg, 'numpy')
        print(num, end='\r')
        vals = [lmb(*list(sub_d.values()))  for sub_d in sub_dict]
        matrix[ind] = np.dot(np.array(vals).T,glob_2d_weights)
    except:
        print(gg, gg.free_symbols)
        raise Exception
    

In [16]:
index = polynom_factory.local_index(dim=2,distribution='globatto', order=order)

In [ ]:
ref_matrix, index = polynom_factory.local_gradgrad_matrix(dim=2,distribution='globatto', order=order)

In [17]:
normed = np.dot(np.diag(1/glob_2d_weights), matrix)

In [18]:
for point, num in index.items():
    partial_diff_test(matrix=normed, point=point, pointnum=num, pointdict=index, multiple_dofs_on_point=False)
    #partial_diff_test(matrix=ref_matrix, point=point, pointnum=num, pointdict=index, multiple_dofs_on_point=False)
    
    print('\n\n')

X derivative: 0.0003255479446808789
Y derivative: -30.000260430181363
XX derivative: -1.000008975122723
YY derivative: -1.000031541270439
XY_derivative: 1.277520184112797e-10




X derivative: 5.8110362030296655e-05
Y derivative: -9.923645033449446e-05
XX derivative: -1.0000457434723848
YY derivative: -1.0000131251649784
XY_derivative: -1.7636356004666384e-09




X derivative: -29.999741256948187
Y derivative: 1.1850130018225493e-05
XX derivative: -0.9999230764090633
YY derivative: -1.0000025880365593
XY_derivative: -1.549661703161235e-08




X derivative: -7.655218390033749e-05
Y derivative: -3.226402349446542e-05
XX derivative: -1.0000377787492547
YY derivative: -1.0000023720158133
XY_derivative: 5.712761034919031e-10




X derivative: 30.00011347904406
Y derivative: 2.5288418004290997e-05
XX derivative: -1.0000450981646036
YY derivative: -0.9999123705454924
XY_derivative: -2.0124237597125524e-09




X derivative: 0.0001702192740956731
Y derivative: -4.850458730186946e-05
XX derivati

In [7]:
f = h5py.File('../datasources/5_6_globatto_matrices.h5', mode='r')

In [8]:
mass_5 = f['mass_matrices']['base_M_order_5'][()]

In [9]:
mass_5_d = np.diag(mass_5.sum(axis=1))

In [10]:
mass_5_flat = mass_5.sum(axis=1)
glob_2d_weights = mass_5_flat

In [20]:
np.save(arr=mass_5_d, file='../datasources/pipka_mass.npz')

In [21]:
np.save(arr=matrix, file='../datasources/pipka.npz')